In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime as dt
import statsmodels as st
import calendar,warnings,itertools,keras,shutil
from datetime import datetime
from keras import models
warnings.filterwarnings('ignore')

#Read data set from environment
dataset=pd.read_csv('DataCoSupplyChainDataset.csv',encoding='unicode_escape',header=0)
#finding shape of dataset
dataset.shape

(180519, 53)

In [2]:
from keras import models
from keras import layers
from keras import regularizers


In [3]:
data=pd.DataFrame(dataset,columns=['Type','Benefit per order','Sales per customer','Late_delivery_risk',
                                  'Category Name','Customer Segment','Department Name','Market',
                                  'Order Country','Order Item Discount','Order Item Discount Rate',
                                  'Order Item Product Price','Order Item Profit Ratio','Order Item Quantity','Order Item Total',
                                  'Order Profit Per Order','Order Region','Order Status','Product Price','order date (DateOrders)','shipping date (DateOrders)'])

In [4]:
data.shape

(180519, 21)

In [5]:
data.head()

,Type,Benefit per order,Sales per customer,Late_delivery_risk,Category Name,Customer Segment,Department Name,Market,Order Country,Order Item Discount,...,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Order Region,Order Status,Product Price,order date (DateOrders),shipping date (DateOrders)
0,DEBIT,91.250000,314.640015,0,Sporting Goods,Consumer,Fitness,Pacific Asia,Indonesia,13.110000,...,327.75,0.29,1,314.640015,91.250000,Southeast Asia,COMPLETE,327.75,1/31/2018 22:56,2/3/2018 22:56
1,TRANSFER,-249.089996,311.359985,1,Sporting Goods,Consumer,Fitness,Pacific Asia,India,16.389999,...,327.75,-0.80,1,311.359985,-249.089996,South Asia,PENDING,327.75,1/13/2018 12:27,1/18/2018 12:27
2,CASH,-247.779999,309.720001,0,Sporting Goods,Consumer,Fitness,Pacific Asia,India,18.030001,...,327.75,-0.80,1,309.720001,-247.779999,South Asia,CLOSED,327.75,1/13/2018 12:06,1/17/2018 12:06
3,DEBIT,22.860001,304.809998,0,Sporting Goods,Home Office,Fitness,Pacific Asia,Australia,22.940001,...,327.75,0.08,1,304.809998,22.860001,Oceania,COMPLETE,327.75,1/13/2018 11:45,1/16/2018 11:45
4,PAYMENT,134.210007,298.250000,0,Sporting Goods,Corporate,Fitness,Pacific Asia,Australia,29.500000,...,327.75,0.45,1,298.250000,134.210007,Oceania,PENDING_PAYMENT,327.75,1/13/2018 11:24,1/15/2018 11:24


In [6]:
train_y=pd.DataFrame(dataset,columns=['Sales'])

In [41]:

train_y.head(150)

,Sales
0,327.750000
1,327.750000
2,327.750000
3,327.750000
4,327.750000
...,...
145,252.880005
146,452.040008
147,252.880005
148,452.040008


In [8]:
df2=pd.DataFrame(data,columns=['Benefit per order','Sales per customer','Late_delivery_risk','Order Item Discount','Order Item Discount Rate',
                              'Order Item Product Price','Order Item Profit Ratio','Order Item Quantity','Order Item Total','Order Profit Per Order',
                              'Product Price','order date (DateOrders)','shipping date (DateOrders)'])

In [9]:
df2.shape

(180519, 13)

In [10]:
df=pd.DataFrame(data,columns=['Type','Category Name','Customer Segment','Department Name','Market','Order Country','Order Region'
                             ,'Order Status'])

In [11]:
df1=pd.get_dummies(df)

In [12]:
import time
from datetime import datetime

In [13]:
df2['order date (DateOrders)']=pd.to_datetime(df2['order date (DateOrders)'])
df2['shipping date (DateOrders)']=pd.to_datetime(df2['shipping date (DateOrders)'])
df2['dates_diff']=df2['shipping date (DateOrders)']-df2['order date (DateOrders)']


In [14]:
df2.dtypes

Benefit per order                     float64
Sales per customer                    float64
Late_delivery_risk                      int64
Order Item Discount                   float64
Order Item Discount Rate              float64
Order Item Product Price              float64
Order Item Profit Ratio               float64
Order Item Quantity                     int64
Order Item Total                      float64
Order Profit Per Order                float64
Product Price                         float64
order date (DateOrders)        datetime64[ns]
shipping date (DateOrders)     datetime64[ns]
dates_diff                    timedelta64[ns]
dtype: object

In [15]:
df1= df1.astype(np.float32)

In [16]:
df1.head()

,Type_CASH,Type_DEBIT,Type_PAYMENT,Type_TRANSFER,Category Name_Accessories,Category Name_As Seen on TV!,Category Name_Baby,Category Name_Baseball & Softball,Category Name_Basketball,Category Name_Books,...,Order Region_Western Europe,Order Status_CANCELED,Order Status_CLOSED,Order Status_COMPLETE,Order Status_ON_HOLD,Order Status_PAYMENT_REVIEW,Order Status_PENDING,Order Status_PENDING_PAYMENT,Order Status_PROCESSING,Order Status_SUSPECTED_FRAUD
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
df2.head()

,Benefit per order,Sales per customer,Late_delivery_risk,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Product Price,order date (DateOrders),shipping date (DateOrders),dates_diff
0,91.250000,314.640015,0,13.110000,0.04,327.75,0.29,1,314.640015,91.250000,327.75,2018-01-31 22:56:00,2018-02-03 22:56:00,3 days
1,-249.089996,311.359985,1,16.389999,0.05,327.75,-0.80,1,311.359985,-249.089996,327.75,2018-01-13 12:27:00,2018-01-18 12:27:00,5 days
2,-247.779999,309.720001,0,18.030001,0.06,327.75,-0.80,1,309.720001,-247.779999,327.75,2018-01-13 12:06:00,2018-01-17 12:06:00,4 days
3,22.860001,304.809998,0,22.940001,0.07,327.75,0.08,1,304.809998,22.860001,327.75,2018-01-13 11:45:00,2018-01-16 11:45:00,3 days
4,134.210007,298.250000,0,29.500000,0.09,327.75,0.45,1,298.250000,134.210007,327.75,2018-01-13 11:24:00,2018-01-15 11:24:00,2 days


In [18]:
df2.shape

(180519, 14)

In [19]:
float("3 days".replace('days',''))

3.0

In [20]:
df3=df2.drop(['order date (DateOrders)','shipping date (DateOrders)'],axis=1)

In [21]:
df3.head(2)

,Benefit per order,Sales per customer,Late_delivery_risk,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Order Item Total,Order Profit Per Order,Product Price,dates_diff
0,91.250000,314.640015,0,13.110000,0.04,327.75,0.29,1,314.640015,91.250000,327.75,3 days
1,-249.089996,311.359985,1,16.389999,0.05,327.75,-0.80,1,311.359985,-249.089996,327.75,5 days


In [22]:
"3 days 00:00:00".split(' ')

['3', 'days', '00:00:00']

In [23]:
days=[]
for day in df3['dates_diff']:
    x=str(day).split(' ')
    days.append(float(x[0]))
    print(x[0])
   

3
5
4
3
2
6
2
2
3
2
6
5
4
2
2
2
5
2
2
0
0
5
4
3
2
6
5
4
3
2
6
5
4
2
2
2
2
2
0
0
0
0
4
3
2
6
5
4
5
2
6
2
3
5
6
2
5
6
4
2
5
5
5
3
2
6
4
2
3
3
2
4
3
6
6
5
4
5
5
5
3
4
4
4
4
5
6
2
4
4
4
4
5
6
4
2
6
2
3
5
4
3
3
3
4
2
4
5
6
4
5
4
3
6
3
5
4
6
4
2
5
4
5
2
5
4
6
6
2
3
2
2
6
3
5
5
5
6
6
3
5
6
4
4
5
3
4
3
6
3
6
6
5
2
3
5
4
2
4
4
5
2
6
4
3
3
6
5
5
4
3
6
5
2
5
5
6
4
5
4
4
5
5
5
5
6
5
2
3
3
2
3
6
5
6
4
2
4
6
2
6
4
4
5
2
2
4
5
5
2
5
6
5
4
5
3
4
2
5
4
5
5
3
3
5
5
2
4
2
2
4
3
2
5
6
4
2
5
6
4
2
4
4
2
4
3
2
4
5
6
4
4
4
5
3
3
4
4
6
3
4
2
4
5
5
4
2
3
4
5
2
2
4
4
2
3
4
6
4
5
3
3
6
5
4
6
5
6
5
2
5
5
3
2
3
2
2
2
6
6
6
6
3
3
6
2
4
2
5
6
4
3
6
2
5
6
5
6
3
6
2
4
6
3
2
4
2
4
3
4
5
6
6
5
5
3
3
5
3
4
4
3
4
5
3
2
3
2
4
2
2
6
5
2
4
2
5
2
6
3
5
6
3
2
6
5
4
3
2
6
5
2
3
2
6
5
4
3
2
6
5
4
3
2
2
2
2
0
0
0
5
4
3
2
6
5
4
3
2
2
5
4
3
2
6
5
4
3
2
6
5
4
3
2
6
5
2
2
2
6
5
2
2
2
6
5
4
3
2
6
5
4
3
2
6
5
4
3
2
6
5
4
3
2
6
5
4
0
0
0
0
0
0
0
6
5
4
3
6
6
2
6
5
3
3
2
2
6
2
5
5
3
5
6
2
5
5
4
6
3
2
6
2
2
4
3
3
2
3
2
4
2
3
5
2
3
2
5
4
3


5
4
4
3
3
5
3
6
3
2
4
4
4
4
3
6
4
6
2
5
5
2
4
6
6
2
3
2
6
3
4
3
5
6
3
6
5
5
5
5
5
6
5
2
5
4
5
4
6
2
2
6
2
4
4
4
2
6
6
2
4
5
2
4
3
3
6
6
2
4
6
3
5
5
4
5
5
5
2
6
4
6
6
5
3
6
4
5
5
4
3
4
4
4
3
3
3
3
5
3
5
4
3
3
6
4
4
4
2
2
3
5
6
2
6
4
5
6
4
4
4
5
6
2
5
4
6
4
6
6
2
5
5
6
5
5
3
5
4
6
5
3
4
2
6
2
4
3
6
6
6
6
4
4
2
2
4
3
4
5
5
2
4
2
2
3
2
5
4
6
4
6
4
4
6
4
3
4
6
5
2
3
5
5
5
4
5
3
3
5
4
2
5
5
4
5
3
4
2
2
2
5
2
2
4
3
2
5
4
6
5
5
5
4
6
5
3
2
6
3
3
3
5
5
6
5
6
3
2
3
4
2
4
2
3
4
3
2
4
2
2
5
6
2
3
3
2
5
5
6
2
5
2
2
6
5
6
4
5
2
6
4
6
2
4
3
2
4
5
5
5
4
3
5
5
6
2
4
3
4
5
5
2
2
4
6
5
2
4
5
6
2
3
2
5
2
4
5
5
4
2
2
2
3
5
3
3
5
6
4
2
3
2
4
2
4
4
6
4
4
2
2
6
4
3
5
2
6
3
3
5
3
6
3
3
3
2
3
6
3
2
4
5
2
4
4
4
3
3
6
3
6
2
6
5
6
4
4
6
2
5
2
2
4
2
6
3
5
3
3
4
3
6
2
5
6
4
6
6
5
2
3
2
3
4
2
2
3
3
3
5
4
2
6
3
6
4
2
2
2
5
2
5
4
3
4
4
2
4
3
4
6
4
4
3
3
2
2
4
2
4
3
3
2
4
3
4
3
5
5
3
6
5
4
3
4
5
4
3
4
6
4
2
2
5
4
3
4
2
5
3
3
3
6
6
3
5
3
6
5
6
6
5
4
2
5
5
3
3
3
3
2
5
3
5
5
6
4
5
2
3
5
2
5
4
5
5
5
5
5
4
6
6
6
4
4
3
6
2
4


3
2
6
4
5
3
6
2
5
4
5
3
6
6
6
4
3
6
2
2
4
6
5
2
2
4
2
3
5
4
4
5
3
4
5
3
5
2
3
5
2
4
6
5
4
6
3
4
5
6
5
5
6
4
6
4
6
5
2
6
5
3
2
2
2
5
3
4
3
4
5
6
5
6
3
5
6
3
3
3
5
5
4
4
2
4
4
2
6
6
5
5
6
3
2
2
4
6
5
2
5
3
2
4
4
5
2
3
3
5
3
3
6
3
2
2
5
4
4
5
5
4
4
4
5
5
6
5
4
5
4
5
4
3
5
6
3
3
4
2
6
2
5
3
2
5
4
2
4
6
3
3
3
4
4
5
3
5
2
2
5
5
5
5
6
5
3
3
5
4
2
6
5
2
2
3
5
4
3
2
3
3
2
3
4
2
5
4
3
2
6
2
2
2
4
6
6
6
2
5
6
2
5
6
3
6
2
5
2
4
6
6
4
5
2
2
5
4
5
6
2
4
3
5
2
2
4
2
5
4
3
3
2
2
4
5
6
2
5
6
6
5
4
6
6
2
3
2
5
3
2
6
5
4
4
5
3
2
5
2
2
4
6
4
2
2
6
3
3
4
3
5
3
6
6
2
2
4
3
4
2
4
4
4
3
6
6
4
5
3
5
4
4
5
4
5
4
3
3
3
6
3
6
6
3
3
2
5
4
2
6
4
6
3
6
5
2
2
6
4
5
2
2
5
4
3
6
3
4
4
3
2
5
3
5
6
6
5
3
6
2
6
4
4
5
3
4
5
4
2
6
5
3
5
5
6
2
3
5
2
4
6
4
6
6
6
6
2
5
2
4
5
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


4
3
3
5
4
2
6
6
3
2
4
2
2
6
6
6
5
6
3
6
3
5
2
2
3
3
4
5
2
3
3
6
5
6
3
5
4
2
3
4
2
4
5
6
6
3
4
3
3
6
4
4
5
2
6
4
6
5
6
6
2
4
4
3
6
4
2
6
5
2
4
3
6
3
4
6
2
3
3
6
2
4
6
3
2
2
3
6
2
3
3
3
3
2
4
6
2
5
6
4
4
2
3
2
2
2
4
6
6
5
2
4
5
3
3
3
2
5
3
5
6
6
2
5
3
2
4
5
5
5
4
3
5
2
2
6
5
2
4
6
2
6
4
6
3
5
3
4
3
5
3
6
2
2
4
4
4
5
3
6
3
2
3
6
3
4
3
4
2
2
6
3
6
6
2
3
3
2
3
2
5
3
4
4
3
3
3
6
5
2
3
4
3
4
4
3
5
5
3
3
3
5
5
5
5
5
3
5
3
5
3
2
3
5
4
5
3
3
2
2
6
6
5
6
3
2
2
6
4
4
4
2
4
4
5
5
3
2
3
3
2
3
5
4
2
5
3
6
6
3
5
4
4
4
5
3
5
4
5
3
3
4
5
6
3
2
3
5
5
2
5
5
4
5
6
6
4
6
3
4
5
3
4
3
5
4
3
6
4
5
5
3
5
3
6
6
3
6
6
2
6
2
2
4
4
5
2
4
4
3
2
2
5
3
2
2
6
3
6
5
5
3
2
5
2
5
5
5
5
2
4
4
5
4
4
6
4
4
5
4
2
4
3
3
4
2
5
4
3
4
6
6
6
2
3
3
5
3
2
3
2
3
6
5
2
5
2
2
2
6
4
6
2
5
4
5
2
2
6
5
6
4
3
3
3
2
6
5
2
3
3
3
6
6
4
2
2
2
3
4
6
6
2
5
5
2
2
3
6
5
3
3
4
5
2
3
5
3
4
5
6
6
4
5
5
3
3
5
6
5
5
5
5
5
2
2
2
2
4
2
2
6
4
6
2
5
6
2
4
2
6
6
5
6
6
3
2
5
6
6
6
5
5
4
6
6
5
5
6
4
6
4
6
4
5
3
5
2
5
3
2
4
6
4
2
2
6
5
4
3
4
6
4
4
4
3
5
2
6
5


5
6
5
5
4
5
4
6
4
5
5
3
5
6
6
4
5
4
6
3
5
4
5
3
2
2
5
5
5
5
5
5
2
3
6
2
5
5
4
2
4
4
5
2
3
6
6
2
6
2
5
2
2
2
4
2
2
5
5
4
6
3
5
3
6
4
4
3
2
4
3
4
3
5
6
6
5
2
2
6
6
3
4
6
6
3
4
2
4
4
2
4
2
2
4
6
6
5
2
5
5
4
3
2
6
3
6
4
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


...

6
5
3
3
3
3
4
2
5
4
6
3
4
6
6
3
3
3
5
4
3
3
3
6
2
4
6
2
5
5
5
3
4
5
5
2
6
6
3
6
2
3
4
3
5
4
6
5
2
2
5
5
5
2
2
6
3
6
5
4
5
3
4
6
3
4
2
6
2
4
3
3
6
3
3
3
6
3
4
5
4
2
2
5
3
5
3
2
3
5
3
2
4
5
5
6
4
5
2
5
2
2
3
2
2
6
5
3
3
6
5
6
3
5
3
5
3
2
3
3
3
2
2
5
2
3
4
3
3
2
6
4
2
4
3
3
6
5
5
5
2
5
4
4
4
5
5
3
6
2
6
6
3
4
6
3
5
6
3
2
4
3
2
6
6
5
3
4
4
2
2
2
2
3
4
4
2
5
5
6
3
5
2
6
2
2
2
5
4
6
6
4
6
4
2
2
3
3
2
6
6
6
5
6
4
2
4
2
6
6
5
2
4
2
4
2
3
5
3
3
4
2
6
6
2
3
6
5
5
2
6
2
3
2
3
6
2
2
2
6
2
2
3
3
2
5
5
6
6
4
4
2
2
5
5
5
3
5
5
4
5
2
2
2
6
3
3
3
3
6
6
2
2
2
2
4
6
5
4
2
2
2
3
3
6
2
3
4
4
2
4
6
2
3
5
5
5
3
6
3
5
4
6
6
3
6
5
3
6
5
2
3
5
2
4
6
2
5
5
5
3
2
3
5
4
6
4
5
2
4
5
2
6
4
4
3
6
4
5
2
3
5
3
3
6
3
3
5
6
5
6
3
2
5
4
6
5
5
6
5
3
6
3
2
4
6
3
4
6
2
5
2
4
4
3
3
5
2
4
4
3
5
2
3
4
3
2
6
6
5
3
5
4
4
2
6
6
2
2
6
4
2
6
3
6
2
4
4
6
6
5
6
4
3
4
4
2
6
6
4
3
4
4
3
4
2
4
6
2
5
3
5
4
6
6
6
6
6
6
3
3
3
6
6
5
5
6
6
6
2
5
6
5
4
4
4
4
2
5
6
3
4
2
4
6
5
6
4
3
3
3
4
5
3
3
4
5
3
5
6
2
2
2
2
3
4
2
2
2
6
2
3
2
6
5
2
4
2
3
5


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
4
5
4
6
6
5
2
2
2
2
6
3
2
5
2
2
2
5
6
6
4
6
4
5
6
6
5
3
6
5
4
6
6
6
6
5
5
2
2
2
6
3
2
5
2
5
3
3
3
3
5
3
3
3
3
2
4
5
3
3
4
4
3
2
2
4
3
6
6
5
2
6
4
5
5
5
6
5
2
3
4
6
5
2
6
4
6
2
3
6
3
4
2
3
2
6
6
2
2
3
2
2
2
5
2
4
5
5
4
6
2
4
5
3
6
3
5
2
5
3
3
4
6
3
6
6
5
4
5
3
2
2
6
2
6
6
5
4
4
3
2
6
6
6
2
3
4
2
3
5
6
4
4
2
5
4
4
4
4
6
4
5
2
2
5
3
5
2
6
2
5
3
6
2
2
3
3
4
4
6
2
2
3
5
6
2
6
6
2
3
6
6
4
5
3
5
4
5
4
5
4
5
5
4
3
2
2
2
2
2
5
2
4
2
4
3
3
4
2
2
2
5
2
5
2
6


6
3
3
3
4
4
6
5
2
4
6
4
5
5
3
2
3
3
3
5
3
4
3
6
4
2
5
2
2
2
3
6
6
6
3
5
3
6
3
3
3
3
2
4
5
6
3
5
3
5
4
5
4
6
4
4
3
4
3
3
2
6
5
4
5
6
6
5
2
2
2
5
6
5
6
5
4
2
3
6
4
5
4
6
5
4
5
4
4
3
4
5
3
5
4
4
5
5
4
3
5
4
4
3
4
5
4
5
5
5
3
6
2
5
4
5
6
4
2
3
6
2
2
2
6
3
3
6
6
6
4
5
4
2
2
3
6
6
2
2
2
3
3
6
6
3
5
3
3
3
3
3
6
2
3
3
6
5
2
3
3
2
6
2
3
6
2
2
4
4
6
2
4
6
4
5
6
3
4
6
2
6
5
2
6
5
5
5
6
5
6
3
3
6
4
3
6
6
6
3
4
4
2
4
4
3
3
6
6
5
6
3
6
5
6
6
3
6
3
6
2
2
6
3
2
2
2
2
5
5
2
6
2
5
5
6
2
3
5
3
3
3
3
6
6
5
5
3
4
4
6
4
3
4
4
6
4
5
5
2
3
6
2
4
4
6
4
2
3
3
4
4
2
5
2
5
2
3
5
5
4
6
6
4
5
4
6
5
5
2
6
2
2
6
3
6
6
2
2
3
5
2
6
4
3
4
5
6
5
6
3
5
2
6
4
4
4
6
2
6
2
2
4
6
3
5
4
6
4
6
2
6
6
3
2
4
2
3
5
2
2
6
2
6
2
4
4
6
6
4
2
6
4
2
2
5
3
4
6
2
4
2
2
6
5
4
3
4
3
2
5
2
2
6
2
2
6
5
2
4
4
2
5
6
5
6
4
4
6
5
4
3
4
4
3
2
4
2
3
4
3
4
2
2
5
5
5
2
2
2
5
4
4
4
5
2
5
2
5
6
2
6
3
6
6
5
6
5
6
5
5
5
5
6
5
6
6
5
2
6
2
3
2
6
6
5
5
2
6
3
5
2
5
6
3
2
5
2
4
5
6
5
4
2
6
2
5
4
5
6
3
4
6
3
5
5
6
4
6
5
6
4
2
3
5
5
2
6
4
4
6
5
2
5
5
6
3
5
4
4


3
2
3
3
3
4
3
3
4
3
6
6
3
5
4
2
3
4
3
6
5
4
3
2
2
6
2
3
3
6
3
6
5
4
6
2
5
5
5
4
5
3
4
5
3
2
3
6
3
3
2
3
4
2
6
4
5
2
3
6
3
3
5
6
4
5
3
4
4
5
2
5
3
2
4
3
2
4
5
3
6
4
5
4
6
4
4
6
6
4
2
3
3
6
3
4
2
5
2
5
2
6
6
2
3
3
4
5
6
3
4
3
3
4
5
5
5
6
6
2
4
5
4
2
2
5
6
2
3
5
2
4
2
2
2
3
5
6
4
2
5
4
4
6
6
4
6
2
4
5
6
2
3
6
5
5
6
6
5
3
3
5
5
3
3
2
2
2
3
2
5
2
6
6
5
3
4
5
2
3
3
3
5
2
2
5
4
6
5
5
3
3
3
3
6
2
6
5
5
5
5
6
4
5
2
4
5
2
6
5
4
6
2
5
4
6
2
2
5
4
5
2
3
5
2
2
6
4
6
5
3
4
4
4
5
5
6
2
2
6
3
4
4
3
3
2
4
6
3
2
4
4
2
4
2
6
5
5
5
2
5
2
4
3
4
4
3
3
6
6
5
5
5
3
5
2
2
5
3
3
3
5
3
6
6
6
3
2
6
6
5
6
4
5
3
5
2
2
4
4
6
2
4
4
5
2
4
2
4
4
6
5
5
5
5
5
6
2
2
2
4
4
2
3
6
6
6
5
2
5
4
5
4
3
5
6
2
6
6
2
3
6
6
5
5
4
5
4
6
6
5
4
5
6
6
6
2
4
3
5
2
4
5
2
3
5
5
2
3
4
5
4
2
2
4
2
3
3
6
6
6
6
2
6
6
2
2
3
6
5
3
2
2
5
6
6
5
4
5
5
3
2
6
5
4
5
2
3
2
4
3
6
3
4
4
5
5
2
4
6
2
4
3
3
3
3
5
6
4
3
5
3
3
4
4
4
5
3
5
2
5
2
4
5
3
3
2
3
4
3
2
3
4
4
3
4
6
2
4
6
2
2
6
6
3
2
6
5
4
5
4
3
2
6
5
3
3
5
3
6
4
3
2
2
4
5
3
6
6
6
5
6
5
3
2
5
5
3
6
3


2
3
3
2
2
5
5
2
6
2
4
4
2
3
5
5
2
2
3
5
2
5
3
2
5
4
6
2
4
4
6
4
5
5
4
5
4
4
6
6
2
4
6
3
6
6
2
5
6
3
6
3
3
5
5
5
6
6
3
4
2
4
2
5
4
2
6
5
4
2
3
3
5
2
4
6
6
4
6
4
6
4
6
3
6
2
2
5
2
5
2
2
5
2
4
3
2
2
2
3
4
4
5
6
5
4
6
3
3
4
2
6
3
3
6
5
5
2
2
5
6
6
5
6
5
5
6
5
5
6
6
3
2
4
6
4
4
5
2
2
6
3
3
6
3
5
2
6
3
5
6
6
6
5
2
5
5
4
5
5
5
5
5
2
2
3
3
2
6
2
5
5
4
5
2
3
2
6
4
5
6
2
6
4
2
4
3
6
6
6
2
6
5
2
6
5
4
4
5
3
2
2
6
4
6
2
3
2
4
2
4
4
4
2
5
3
3
5
6
3
2
2
5
5
5
4
6
5
5
3
6
6
2
5
3
2
3
3
3
4
3
2
2
3
3
2
2
3
5
4
2
5
5
2
2
5
5
2
2
5
6
4
4
5
4
2
3
4
3
5
3
4
6
3
3
3
2
2
5
3
5
3
4
6
3
2
4
6
5
4
3
5
6
2
4
6
4
6
4
3
5
6
4
6
4
4
4
5
5
6
5
6
6
6
4
4
5
3
2
5
4
4
4
3
6
4
3
4
6
3
2
4
3
3
5
6
2
4
2
4
5
5
3
3
5
4
4
2
2
6
5
6
6
5
2
4
2
3
5
4
4
4
5
6
4
6
4
2
6
2
4
6
2
2
3
2
4
4
2
5
2
2
4
4
6
4
3
5
2
2
2
2
2
5
4
2
3
6
2
5
2
3
4
2
5
5
4
5
2
4
5
2
3
4
4
2
6
4
5
4
3
2
3
2
2
5
4
2
4
2
2
5
2
2
2
5
2
5
6
3
6
4
4
5
3
5
3
3
2
4
4
4
6
6
3
3
4
5
5
3
5
6
5
6
6
6
3
4
4
6
6
2
5
6
2
2
4
2
3
5
5
2
3
5
4
3
3
4
5
5
6
3
4
5
6
2
5
3
4
2


In [24]:
len(days)

180519

In [25]:
df3['dates_diff']=days

In [26]:
df3.dtypes

Benefit per order           float64
Sales per customer          float64
Late_delivery_risk            int64
Order Item Discount         float64
Order Item Discount Rate    float64
Order Item Product Price    float64
Order Item Profit Ratio     float64
Order Item Quantity           int64
Order Item Total            float64
Order Profit Per Order      float64
Product Price               float64
dates_diff                  float64
dtype: object

In [39]:
df3.to_csv('file2.csv', header=False, index=False)

In [28]:
merge=[df1,df3]
train_x=pd.concat(merge,axis=1)

In [29]:
train_x.shape

(180519, 281)

In [30]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU

In [31]:
model=Sequential()

In [32]:

leaky_relu_alpha = 0.1

In [33]:
model.add(Dense(150,activation='relu',input_shape=[281]))
model.add(Dense(150,activation='relu'))
model.add(Dense(3,))

In [34]:
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.utils import np_utils

In [35]:
model.compile(loss='MeanSquaredError',
              optimizer=keras.optimizers.Adam())

In [36]:
model.fit(train_x,train_y,validation_split=0.2,epochs=10)

Epoch 1/10
4513/4513 [==============================] - 11s 3ms/step - loss: 94.7175 - val_loss: 0.1307
Epoch 2/10
4513/4513 [==============================] - 12s 3ms/step - loss: 5.2510 - val_loss: 0.6792
Epoch 3/10
4513/4513 [==============================] - 12s 3ms/step - loss: 1.7909 - val_loss: 0.0191
Epoch 4/10
4513/4513 [==============================] - 11s 2ms/step - loss: 1.5209 - val_loss: 0.0300
Epoch 5/10
4513/4513 [==============================] - 11s 2ms/step - loss: 5.3818 - val_loss: 0.0097
Epoch 6/10
4513/4513 [==============================] - 12s 3ms/step - loss: 1.5825 - val_loss: 0.6393
Epoch 7/10
4513/4513 [==============================] - 12s 3ms/step - loss: 2.0400 - val_loss: 0.0080
Epoch 8/10
4513/4513 [==============================] - 12s 3ms/step - loss: 0.7342 - val_loss: 0.2019
Epoch 9/10
4513/4513 [==============================] - 11s 3ms/step - loss: 4.8183 - val_loss: 0.0212
Epoch 10/10
4513/4513 [==============================] - 12s 3ms/step - 

In [37]:
import pickle


In [38]:

model.save('model_weights.h5')